In [ ]:
!pip install llama-index
!pip install llama-index-llms-together
!pip install python-dotenv

In [ ]:
!pip freeze > requirements.txt

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os
from typing import List, Optional
from llama_index.llms.together import TogetherLLM
from llama_index.core.agent.workflow import FunctionAgent
API_TOKEN: Optional[str] = os.getenv("LLM_API_TOKEN")

### Basic together.ai example

In [13]:
llm = TogetherLLM(
    # DONT CHANGE THE MODEL
    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    api_key=API_TOKEN
)

In [14]:
resp = llm.complete("WHAT THE")
print(resp)

<think>

</think>

It seems like your message got cut off. Could you clarify or provide more details about what you're asking or referring to? I'm here to help! 😊


# Overall Workflow (i think?)
1. Manifesto data > PDF Data Connector (or any applicable connector)
2. Structured Data Extraction on manifesto to issues/questions
3. Convert extracted data into SQL and then insert into database

Then, augment model with RAG from database -> Make model have access to our issue/stances database so that it can see the current issues/stances and fill in gaps or ignore if we already have the data

## Pydantic Classes for SDE

In [3]:
from pydantic import BaseModel, Field

# Need to somehow cross check category
class Issue(BaseModel):
    """An issue, problem, or event that can be addressed by political parties, typically for them to form policy around."""
    
    description: str = Field(description="A long (max 300 characters) description of the issue at hand")
    summary: str = Field(description="A short (max 50 characters), concise, summary of the issue")

class Stance(BaseModel):
    """A political party's stance on an issue"""
    
    issue: Issue = Field(description = "The issue that is being discussed")
    stand: bool = Field(description = "Whether the party disagrees or agrees with the contents of the issue")
    reason: str = Field(description = "A description on why the bparty has this stance")
    party: str = Field(description = "The name of the political party that holds this stance")    

## Reading WP manifesto data (from .html)

In [4]:
from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(
    input_files=["./data/WP-Manifesto.html"]
)
data = reader.load_data()
text = data[0].text

## Instantiating the Structured LLM

In [7]:
llm = TogetherLLM(
    # DONT CHANGE THE MODEL
    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    api_key=API_TOKEN,
    tools=[]
)
sllm = llm.as_structured_llm(Issue)

## Extracting the data

In [8]:
response = sllm.complete(text)

BadRequestError: Error code: 400 - {'id': 'o2jZuGi-2j9zxn-95de1ead8e4efd77', 'error': {'message': 'invalid tools', 'type': 'invalid_request_error', 'param': 'tools', 'code': None}}